<a href="https://colab.research.google.com/github/aditi230/Analyzing-Product/blob/main/Analyzing_Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install turicreate

In [ ]:
import turicreate

# **READ DATA**

In [ ]:
!unzip amazon_baby.sframe.zip

Archive:  amazon_baby.sframe.zip
   creating: amazon_baby.sframe/
  inflating: amazon_baby.sframe/m_bfaa91c17752f745.frame_idx  
   creating: __MACOSX/
   creating: __MACOSX/amazon_baby.sframe/
  inflating: __MACOSX/amazon_baby.sframe/._m_bfaa91c17752f745.frame_idx  
  inflating: amazon_baby.sframe/m_bfaa91c17752f745.0000  
  inflating: __MACOSX/amazon_baby.sframe/._m_bfaa91c17752f745.0000  
  inflating: amazon_baby.sframe/dir_archive.ini  
  inflating: __MACOSX/amazon_baby.sframe/._dir_archive.ini  
  inflating: amazon_baby.sframe/m_bfaa91c17752f745.sidx  
  inflating: __MACOSX/amazon_baby.sframe/._m_bfaa91c17752f745.sidx  
 extracting: amazon_baby.sframe/objects.bin  
  inflating: __MACOSX/amazon_baby.sframe/._objects.bin  
  inflating: __MACOSX/._amazon_baby.sframe  


In [ ]:
products = turicreate.SFrame('amazon_baby.sframe')

# **LETS EXPLORE**

In [ ]:
products.head

<bound method SFrame.head of Columns:
	name	str
	review	str
	rating	float

Rows: 183531

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Bab

In [ ]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [ ]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [ ]:
def awesome_count(x):
  if 'awesome' in x:
    return x.get('awesome')
  else:
    return 0

In [ ]:
products['awesome'] = products['word_count'].apply(awesome_count)

In [ ]:
for word in selected_words:
  products[word] = products['word_count'].apply(lambda x: x[word] if word in x else 0)

In [ ]:
print('Awesome',products['awesome'].sum())
print('Great',products['great'].sum())
print('Fantastic',products['fantastic'].sum())
print('Amazing',products['amazing'].sum())
print('Love',products['love'].sum())
print('Horrible',products['horrible'].sum())
print('Bad',products['bad'].sum())
print('Terrible',products['terrible'].sum())
print('Awful',products['awful'].sum())
print('Wow',products['wow'].sum())
print('Hate',products['hate'].sum())


Awesome 4075.0
Great 59536.0
Fantastic 1765.0
Amazing 2726.0
Love 43867.0
Horrible 1245
Bad 4950.0
Terrible 1282
Awful 753
Wow 461
Hate 1285


In [ ]:
products = products[products['rating'] != 3]
products['sentiments'] = products['rating'] >=4

In [ ]:
train_data,test_data = products.random_split(.8,seed=0)

In [ ]:
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiments',features=selected_words,validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 1.674086     | 0.847708          | 0.846385            |

| 5         | 6        | 1.828452     | 0.847708          | 0.846385            |

| 6         | 7        | 1.980183     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [ ]:
selected_words_model.coefficients.sort(key_column_names='value',ascending=True)

name,index,class,value,stderr
horrible,None,1,-2.251335236759124,0.08020249388788399
terrible,None,1,-2.2236614360851554,0.07731736203785729
awful,None,1,-2.05290820403138,0.1009973543525922
hate,None,1,-1.3484407222463402,0.07715698604297318
bad,None,1,-0.9914778800650894,0.03848428664699065
wow,None,1,-0.00953823606771224,0.1604641122471162
great,None,1,0.8630655001195999,0.018955052444380473
fantastic,None,1,0.8858047568813963,0.11167591293399713
amazing,None,1,1.1000933113659914,0.09954776260465965
awesome,None,1,1.1335346660341103,0.08399643983187567


In [ ]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6935096220934976,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.3962265467087835,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 5328 

In [ ]:
# Extract only the relevant data
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']
diaper_champ_reviews

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'convenient': 1.0,'more': 1.0, 'trash': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'system': 1.0, 'try':1.0, 're': 1.0, 'still': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'nose': 1.0, 'for': 2.0,'investment': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'out': 1.0, 'pull': 1.0,'open': 1.0, 'pail': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'winter': 1.0,'outside': 1.0, 'day': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'mom': 1.0, 'my': 1.0,'empty': 2.0, 'poop': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'yuck': 1.0, 'clean':1.0, 'all': 1.0, 'tra ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0,'suggestions': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'best': 1.0, 'that':1.0, 'handle': 1.0, ' ...",0.0,0.0,0.0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'no': 1.0, 'regular':1.0, 'part': 1.0, ...",0.0,0.0,0.0


In [ ]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.7919288370624551]

In [ ]:
diaper_champ_reviews['predicted_sentiment'] = selected_words_model.predict(diaper_champ_reviews, output_type = 'probability')

In [ ]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

# **WORD COUNT VECTOR FOR EACH REVIEW**

In [ ]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [ ]:
products.head

In [ ]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,723
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",547
Baby Einstein Take AlongTunes ...,526
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,519
Cloud b TwilightConstellation Night ...,490
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",473
Fisher-Price RainforestJumperoo ...,437
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",386
Leachco Snoogle TotalBody Pillow ...,366
"Regalo Easy Step WalkThru Gate, White ...",333


# **EXPLORE VULLI SOPHIE**

In [ ]:
giraf_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [ ]:
len(giraf_reviews)

In [ ]:
giraf_reviews['rating'].show()

# **BUILD A SENTIMENT CLASSIFIER**

In [ ]:
products['rating'].show()

# **DEFINE POSITIVE AND NEGATIVE SENTIMENTS**

In [ ]:
#IGNORE ALL THRE STAR REVIEWS
products = products[products['rating'] != 3]

In [ ]:
#positive sentiments = 4 or 5 star reviews
products['sentiments'] = products['rating'] >=4

In [ ]:
products

# **LETS TRAIN THE SENTIMENT CLASSIFIER**

In [ ]:
train_data,test_data = products.random_split(.8,seed=0)

In [ ]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiments',features=['word_count'],validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 4.275561     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 4.901182     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 5.855774     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 9.272097     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# **EVALUATE THE SENTIMENT MODEL**

In [ ]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9258242975424673,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.33047871872209655,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0 

In [ ]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']
sentiment_model.predict(diaper_champ_reviews[0:1], output_type='probability')
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type = 'probability')
diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'key': 1.0, 'have': 1.0,'pieces': 1.0, 'betwe ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'priceless': 1.0,'knows': 1.0, 'parent': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0,'suggestions': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'around': 1.0, 'any':1.0, 't': 1.0, 'isn': ...",0.0,1.0,0.0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'either': 1.0, 'be':1.0, 't': 1.0, 'not': ...",0.0,1.0,0.0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'hope': 1.0, 'make':1.0, 'slower': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'buy': 1.0, 'product':1.0, 'recommend': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'ll': 1.0, 'baby': 1.0,'recommended': 1.0, ' ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'very': 1.0, 'told':1.0, 'all': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'sorry': 1.0, 'be': 1.0,'you': 2.0, 'sell': 1.0, ...",0.0,0.0,0.0


# **APPLY THE LEARNED MODEL TO UNDERSTAND SENTIMENT FOR GIRAF**

In [ ]:
giraf_reviews['predicted_sentiment'] = sentiment_model.predict(giraf_reviews,output_type='probability')

In [ ]:
giraf_reviews

# **SORT THE REVIEWS BASED ON THE PREDICTED SENTIMENT AND EXPLORE**

In [ ]:
giraf_reviews = giraf_reviews.sort('predicted_sentiment',ascending=False)

In [ ]:
giraf_reviews

In [ ]:
giraf_reviews[2]['review']

In [ ]:
giraf_reviews[1]['review']

# **SHOW MOST NEGATIVE**

In [ ]:
giraf_reviews[-1]['review']

In [ ]:
giraf_reviews[-2]['review']